# [NLP Project] Word Embedding 기반 양방향 GRU 뉴스 카테고리 분류 모델 프로젝트
- Word2Vec, FastText, GloVe 임베딩의 성능 및 특성 비교 분석

## 환경설정

In [27]:
import os # 파일 시스템
import re # 정규 표현식
import logging # 로깅
import numpy as np # 수치 계산
import pandas as pd # 데이터 처리
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 데이터셋 로드 및 전처리
from sklearn.datasets import load_files # 데이터셋 로드
from sklearn.model_selection import train_test_split # 데이터셋 분할
from nltk.tokenize import word_tokenize # 토큰화
from nltk.corpus import stopwords # 불용어 제거
import nltk # 자연어 처리

# 임베딩 및 시각화
from gensim.models import Word2Vec, FastText # 단어 벡터화 임베딩 모델
import plotly.express as px # 시각화
import plotly.graph_objects as go # 시각화

In [28]:
# ==========================================
# Logger 설정
# ==========================================
logger = logging.getLogger("NLP_Project") # "NLP_Project"라는 이름의 기록관 생성
logger.setLevel(logging.INFO) # 로깅 레벨 설정 (INFO 이상의 메시지만 출력)

# 포맷터 설정 (시간 - 이름 - 레벨 - 메시지)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s') 

# 핸들러 설정
stream_handler = logging.StreamHandler() # 로그를 콘솔에 출력
stream_handler.setFormatter(formatter) # 핸들러에게 "이 디자인(formatter)을 써라"고 명령
if not logger.handlers:                # 중복 방지 (노트북 셀을 여러 번 실행할 때 로그가 여러 줄씩 찍히는 걸 막아줌)
    logger.addHandler(stream_handler)  # 로거에 핸들러를 장착

In [ ]:
# NLTK 리소스 다운로드 (로그 출력 억제를 위해 별도 처리 가능)
nltk.download('punkt', quiet=True) # 토큰화를 위한 리소스 다운로드
nltk.download('stopwords', quiet=True) # 불용어를 위한 리소스 다운로드

True

In [30]:
# ==========================================
# Device 설정 (Apple Silicon MPS 대응)
# ==========================================
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

logger.info(f"Current Device: {device}")

2026-01-08 01:43:43,180 - NLP_Project - INFO - Current Device: mps


# EDA

In [31]:
# ==========================================
# Data Loading
# ==========================================
# 프로젝트 루트 기준 상대 경로
data_dir = "../data"

logger.info("Loading dataset from local path (Encoding: latin1)...")

try:
    train_raw = load_files(os.path.join(data_dir, '20news-bydate-train'), encoding='latin1') # load_files 활용, 데이터 속성 부여 및 리스트화
    test_raw = load_files(os.path.join(data_dir, '20news-bydate-test'), encoding='latin1')

    # 병합
    texts = train_raw.data + test_raw.data
    labels = list(train_raw.target) + list(test_raw.target)
    target_names = train_raw.target_names

    logger.info(f"Dataset loaded successfully.")
    logger.info(f"Total documents: {len(texts)}")
    logger.info(f"Number of categories: {len(target_names)}")
    
    print("\n=== Raw Data Sample (First 1000 chars) ===")
    print(texts[0][:1000]) 
    print("===========================================")

except Exception as e:
    logger.error(f"Error loading dataset: {e}")

2026-01-08 01:43:43,191 - NLP_Project - INFO - Loading dataset from local path (Encoding: latin1)...
2026-01-08 01:43:46,469 - NLP_Project - INFO - Dataset loaded successfully.
2026-01-08 01:43:46,469 - NLP_Project - INFO - Total documents: 18846
2026-01-08 01:43:46,469 - NLP_Project - INFO - Number of categories: 20



=== Raw Data Sample (First 1000 chars) ===
From: cubbie@garnet.berkeley.edu (                               )
Subject: Re: Cubs behind Marlins? How?
Article-I.D.: agate.1pt592$f9a
Organization: University of California, Berkeley
Lines: 12
NNTP-Posting-Host: garnet.berkeley.edu


gajarsky@pilot.njin.net writes:

morgan and guzman will have era's 1 run higher than last year, and
 the cubs will be idiots and not pitch harkey as much as hibbard.
 castillo won't be good (i think he's a stud pitcher)

       This season so far, Morgan and Guzman helped to lead the Cubs
       at top in ERA, even better than THE rotation at Atlanta.
       Cubs ERA at 0.056 while Braves at 0.059. We know it is early
       in the season, we Cubs fans have learned how to enjoy the
       short triumph while it is still there.



> 글 주제, 보내는 곳 등 불필요한 정보 제거 필요 확인

In [ ]:
# ==========================================
# Category Distribution Visualization
# ==========================================
logger.info("Starting EDA: Visualizing category distribution.")

# 시각화를 위한 데이터프레임 구성
df_eda = pd.DataFrame({
    'category': [target_names[i] for i in labels]
})

# 카테고리별 빈도 계산
category_counts = df_eda['category'].value_counts().reset_index()
category_counts.columns = ['category', 'count']

# Plotly Interactive Bar Chart
fig = px.bar(category_counts, 
             x='category', 
             y='count',
             title='20 Newsgroups Category Distribution',
             labels={'category': 'News Category', 'count': 'Document Count'},
             color_continuous_scale='Viridis')

fig.update_layout(xaxis_tickangle=-45, showlegend=False) # 카테고리 이름이 겹치지 않도록 가독성 고려 설정
fig.show()

logger.info("EDA visualization completed.")

2026-01-08 01:43:46,478 - NLP_Project - INFO - Starting EDA: Visualizing category distribution.


2026-01-08 01:43:46,709 - NLP_Project - INFO - EDA visualization completed.


- 대체로 900 ~ 1000개 내외로 카테고리간 데이터 균일
- 하위 3개 카테고리는 상대적으로 데이터가 부족하여, 관련 카테고리에서 오답률이 높아질 수 있음

In [33]:
# [패턴 탐색 코드 블록]
from collections import Counter

logger.info("Analyzing common header patterns...")

# 모든 문서의 각 줄(line) 시작 부분만 추출해서 빈도 계산
header_candidates = []
for doc in texts[:2000]: # 속도를 위해 상위 2000개만 샘플링
    lines = doc.split('\n')
    for line in lines:
        # "단어: " 형태의 패턴을 찾음
        match = re.match(r'^([A-Za-z-]+):', line)
        if match:
            header_candidates.append(match.group(1))

# 가장 많이 등장하는 상위 15개 헤더 키워드 출력
common_headers = Counter(header_candidates).most_common(15)
print("=== Top Detected Header Keywords ===")
for head, count in common_headers:
    print(f"{head}: {count} times")

2026-01-08 01:43:46,719 - NLP_Project - INFO - Analyzing common header patterns...


=== Top Detected Header Keywords ===
Subject: 2037 times
From: 2013 times
Lines: 2002 times
Organization: 1929 times
Distribution: 422 times
NNTP-Posting-Host: 417 times
Nntp-Posting-Host: 409 times
Reply-To: 301 times
Keywords: 165 times
X-Newsreader: 89 times
Summary: 67 times
Originator: 57 times
In-Reply-To: 40 times
Q: 37 times
A: 37 times


> 'Q:', 'A' 는 뉴스 본문 내용 가능성이 있으므로 아래에서 제외하는 헤더 패턴에 포함하지 않음

In [34]:
# NLTK 불용어 리스트 명시적 정의
stop_words = set(stopwords.words('english'))

def clean_text_final(text):
    """
    20 Newsgroups 최적화 정제 함수
    """
    # 1. 헤더 패턴이 포함된 '줄' 전체를 삭제 (모델 커닝, Overfitting 방지) 
    header_pattern = r'^(From|Subject|Lines|Organization|Reply-To|Nntp-Posting-Host|Article-I.D.|Summary|Keywords|Expires|Distribution|Followup-To|X-Newsreader|Originator|In-Reply-To):.*$'
    text = re.sub(header_pattern, '', text, flags=re.IGNORECASE | re.MULTILINE) 
    # re.IGNORECASE: 대소문자 구분 없이 일치하는 패턴 찾기
    # re.MULTILINE: 엔터로 나뉜 모든 줄의 시작을 인식, 본문에 섞인 헤더 줄 제거
    
    # 2. 이메일 주소 제거
    text = re.sub(r'\S+@\S+', '', text)
    
    # 3. 알파벳을 제외한 특수문자 및 숫자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 4. 소문자화 및 토큰화
    tokens = word_tokenize(text.lower())
    
    # 5. 불용어 제거 (정의된 stop_words 사용)
    cleaned_tokens = [w for w in tokens if w not in stop_words and len(w) > 2]
    
    return cleaned_tokens

logger.info("Starting final integrated text cleaning...")

# 전체 데이터에 적용
tokenized_texts = [clean_text_final(doc) for doc in texts]

# 전처리 후 빈 문서 제거 (안전장치)
valid_indices = [i for i, tokens in enumerate(tokenized_texts) if len(tokens) > 0]
tokenized_texts = [tokenized_texts[i] for i in valid_indices]
labels = [labels[i] for i in valid_indices]

logger.info(f"Total documents after cleaning: {len(tokenized_texts)}")

2026-01-08 01:43:46,781 - NLP_Project - INFO - Starting final integrated text cleaning...
2026-01-08 01:43:57,743 - NLP_Project - INFO - Total documents after cleaning: 18817


> 전처리 후 빈 문서 제거 : 모든 텍스트 정제 과정을 거친 후, 유의미한 단어가 하나도 남지 않은 문서는 학습에서 제외하여 모델의 안정성을 높임

In [ ]:
# ==========================================
# 정제 전/후 텍스트 및 길이 비교
# ==========================================
import random

logger.info("Comparing raw text vs cleaned tokens...")

# 1. 무작위 샘플 하나 선택 (검증용)
sample_idx = random.randint(0, len(valid_indices)-1)
raw_idx = valid_indices[sample_idx]

print("\n" + "="*30 + " [Raw Text Sample] " + "="*30)
print(texts[raw_idx][:500] + "...") 

print("\n" + "="*30 + " [Cleaned Tokens Sample] " + "="*30)
print(tokenized_texts[sample_idx][:50]) # 상위 50개 토큰만 출력
print("="*80 + "\n")

# 2. 정제 전/후 길이 변화 시각화 (Plotly)
# 정제 전 길이는 단순 공백 기준 단어 수로 가정하여 비교
raw_lens = [len(str(t).split()) for t in texts]
clean_lens = [len(t) for t in tokenized_texts]

fig_compare = go.Figure()
fig_compare.add_trace(go.Box(y=raw_lens, name='Raw (Word Count)', marker_color='#EF553B'))
fig_compare.add_trace(go.Box(y=clean_lens, name='Cleaned (Token Count)', marker_color='#00CC96'))

fig_compare.update_layout(
    title='Comparison of Document Lengths (Before vs After Cleaning)',
    yaxis_type="log", # 6만 개가 넘는 Max값 때문에 로그 스케일 권장
    yaxis_title="Count (Log Scale)",
    template="plotly_white"
)
fig_compare.show()

2026-01-08 01:43:57,786 - NLP_Project - INFO - Comparing raw text vs cleaned tokens...



============================== [Raw Text Sample] ==============================
From: tysoem@facman.ohsu.edu (Marie E Tysoe)
Subject: Natural Alternatives to Estrogen
Organization: Oregon Health Sciences University
Lines: 2
Nntp-Posting-Host: facman

Need Diet for Diverticular Disease
and ideas for gastrointestinal distress
...

============================== [Cleaned Tokens Sample] ==============================
['need', 'diet', 'diverticular', 'disease', 'ideas', 'gastrointestinal', 'distress']



In [36]:
# 문서 길이 통계 계산
doc_lens = [len(t) for t in tokenized_texts] # 전처리 끝난 토큰화된 텍스트의 길이
p95 = np.quantile(doc_lens, 0.95)
mean_v = np.mean(doc_lens)
max_v = max(doc_lens)

logger.info(f"Statistics - Mean: {mean_v:.2f}, 95th Percentile: {p95:.2f}, Max: {max_v}")

# MAX_LEN 설정
MAX_LEN = int(p95)
logger.info(f"MAX_LEN is set to {MAX_LEN}")

# 1. 히스토그램 생성
fig = px.histogram(x=doc_lens, 
                   nbins=100,
                   title='Distribution of Document Lengths (Token Count)',
                   labels={'x': 'Number of Words', 'y': 'Frequency'},
                   color_discrete_sequence=['skyblue'],
                   opacity=0.7)

# 2. 통계 지표(평균, P95)를 세로선으로 추가
fig.add_vline(x=mean_v, line_dash="dash", line_color="red", 
              annotation_text=f"Mean: {mean_v:.1f}", 
              annotation_position="top left")

fig.add_vline(x=p95, line_dash="solid", line_color="green", 
              annotation_text=f"95th Percentile (MAX_LEN): {p95:.1f}", 
              annotation_position="top right")

# 3. 레이아웃 미세 조정
fig.update_layout(
    showlegend=False,
    xaxis_title="Number of Words",
    yaxis_title="Frequency",
    # 아주 긴 꼬리(Max 6202) 때문에 그래프가 뭉개져 보일 수 있으므로 
    # 시각적 확인을 위해 x축 범위를 p95의 2.5배 정도로 제한 (필요시 조절)
    xaxis_range=[0, p95 * 2.5] 
)

fig.show()

2026-01-08 01:43:58,276 - NLP_Project - INFO - Statistics - Mean: 129.01, 95th Percentile: 358.00, Max: 6202
2026-01-08 01:43:58,277 - NLP_Project - INFO - MAX_LEN is set to 358


> 히스토그램 확인 결과, 데이터가 오른쪽으로 매우 길게 늘어진(Right-skewed) 분포를 보임   
> Max값인 6202를 사용할 경우 과도한 패딩으로 인한 메모리 낭비가 심하므로, 데이터의 95%를 보존하는 358을 최적의 MAX_LEN으로 결정함

# 데이터 분할 (Train/Val/Test)

기존 데이터 셋에 이미 이미 '날짜'를 기준으로 Train과 Test를 나눠 두었으나, 재분할 진행

**데이터 병합 및 재분할의 이유**:
> 1. 제작자가 나눈 시점(날짜) 기반 분할은 특정 시기 단어에 편향될 수 있음.
> 2. 전체 데이터를 섞음(Shuffling)으로써 모델의 일반화 성능을 높임.
> 3. `stratify` 옵션을 사용하여 20개 카테고리의 비율을 Train/Test에 동일하게 유지함.

In [ ]:
# ==========================================
# Train/Validation/Test Split
# ==========================================

# 1단계: 전체 데이터를 Train_Full(80%)과 Test(20%)로 나눔
X_train_full, X_test, y_train_full, y_test = train_test_split(
    tokenized_texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# 2단계: Train_Full을 다시 Train(80%)과 Val(20%)로 나눔
# 결과적으로 전체의 64%가 Train, 16%가 Val, 20%가 Test가 됩니다.
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

logger.info("Data split completed (3-way):")
logger.info(f"- Train size: {len(X_train)}")
logger.info(f"- Val size: {len(X_val)}")
logger.info(f"- Test size: {len(X_test)}")

2026-01-08 01:43:58,721 - NLP_Project - INFO - Data split completed (3-way):
2026-01-08 01:43:58,721 - NLP_Project - INFO - - Train size: 12042
2026-01-08 01:43:58,722 - NLP_Project - INFO - - Val size: 3011
2026-01-08 01:43:58,722 - NLP_Project - INFO - - Test size: 3764


In [38]:
# 전체 카테고리 비율 확인

import pandas as pd

def get_dist(y):
    """타겟 데이터의 클래스별 비율을 반환합니다."""
    return pd.Series(y).value_counts(normalize=True).sort_index()

dist_df_all = pd.DataFrame({
    'Full (%)': pd.Series(labels).value_counts(normalize=True).sort_index() * 100,
    'Train (%)': get_dist(y_train) * 100,
    'Val (%)': get_dist(y_val) * 100,
    'Test (%)': get_dist(y_test) * 100
})

print("=== 20개 카테고리 전체 분포 일치성 확인 (%) ===")
# 소수점 2자리까지 출력해서 가독성 확보
print(dist_df_all.round(2)) 

# 최대 편차 다시 확인 (안심용)
max_diff = (dist_df_all['Full (%)'] - dist_df_all['Train (%)']).abs().max()
logger.info(f"데이터셋 간 최대 비율 편차: {max_diff:.4f}%")

2026-01-08 01:43:58,775 - NLP_Project - INFO - 데이터셋 간 최대 비율 편차: 0.0092%


=== 20개 카테고리 전체 분포 일치성 확인 (%) ===
    Full (%)  Train (%)  Val (%)  Test (%)
0       4.24       4.24     4.25      4.25
1       5.15       5.16     5.15      5.15
2       5.22       5.22     5.21      5.21
3       5.20       5.20     5.21      5.21
4       5.10       5.10     5.08      5.10
5       5.24       5.24     5.25      5.23
6       5.14       5.14     5.15      5.15
7       5.26       5.26     5.25      5.26
8       5.29       5.29     5.28      5.29
9       5.28       5.27     5.28      5.29
10      5.31       5.31     5.31      5.31
11      5.27       5.26     5.28      5.26
12      5.23       5.23     5.21      5.23
13      5.26       5.26     5.25      5.26
14      5.24       5.24     5.25      5.23
15      5.30       5.30     5.31      5.29
16      4.84       4.83     4.85      4.84
17      5.00       5.00     4.98      4.99
18      4.11       4.11     4.12      4.12
19      3.34       3.35     3.32      3.32


**클래스 분포 검증 결과**:
> * 모든 데이터 분할 세트(Train/Val/Test)에서 각 카테고리의 비율이 원본 데이터와 일치함을 확인.
> * 이는 `stratify` 전략이 성공적으로 적용되었음을 의미하며, 학습 결과의 신뢰성을 보장하는 객체적인 근거가 됨.

# 임베딩 모델 설계 및 행렬 구축

In [39]:
# 임베딩 모델 관련 하이퍼파라미터 설정
EMBEDDING_DIM = 100  # 모든 임베딩(W2V, FT, GloVe)과 GRU 입력 차원에 공통 적용
WINDOW_SIZE = 5
MIN_COUNT = 2
epochs = 10

### Word2vec

In [ ]:
import os
import sys
import contextlib
import logging
import warnings
from gensim.models import Word2Vec

# 1. 경고 및 로거 설정
warnings.filterwarnings("ignore")
logging.getLogger("gensim").setLevel(logging.ERROR)

# 2. 시스템 에러(stderr)를 잠시 무시하는 함수
@contextlib.contextmanager
def suppress_stderr():
    with open(os.devnull, "w") as devnull:
        old_stderr = sys.stderr
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stderr = old_stderr

# ==========================================
# Word2Vec 모델 학습 (에러 차단)
# ==========================================

logger.info(f"Training Word2Vec model (CBOW, Epochs: {epochs})...")

with suppress_stderr():
    w2v_model = Word2Vec(
        sentences=X_train, 
        vector_size=EMBEDDING_DIM, 
        window=WINDOW_SIZE, 
        min_count=MIN_COUNT, 
        workers=4,
        sg=0, 
        epochs=epochs
    )

logger.info("Word2Vec training completed.")
logger.info(f"Vocabulary size: {len(w2v_model.wv)}")

# 결과 확인
for word in ['space', 'computer']:
    if word in w2v_model.wv:
        print(f"[{word}] 유사 단어: {w2v_model.wv.most_similar(word, topn=5)}")

2026-01-08 01:43:58,855 - NLP_Project - INFO - Training Word2Vec model (CBOW, Epochs: 10)...
2026-01-08 01:44:05,362 - NLP_Project - INFO - Word2Vec training completed.
2026-01-08 01:44:05,363 - NLP_Project - INFO - Vocabulary size: 47919


[space] 유사 단어: [('advertising', 0.7411476969718933), ('shuttle', 0.7359981536865234), ('rgbyuv', 0.7272762060165405), ('nasa', 0.724191427230835), ('billboard', 0.7004090547561646)]
[computer] 유사 단어: [('shopper', 0.7628679275512695), ('emmisions', 0.6777597665786743), ('architecture', 0.67598956823349), ('overscan', 0.6589229106903076), ('engineering', 0.655694305896759)]


모델 설정: CBOW (sg=0), 100차원, 10회 반복 학습(Epochs=10)  

학습 결과: 총 47,919개의 고유 단어에 대한 벡터 공간 구축 완료   

- space → nasa, shuttle, launch 등 핵심 도메인 단어들이 상위권에 배치되어 기초적인 의미 관계가 형성
- 다만, advertising, shopper 등 당시 뉴스 데이터에 빈번했던 상업적 키워드들이 혼재되어 있음
- 해당 임베딩을 고정(Frozen)하지 않고, 이후 분류 모델 학습 시 **미세 조정(Fine-tuning)**을 통해 노이즈를 스스로 극복하도록 설계

In [ ]:
import numpy as np

# ==========================================
# 정수 인코딩 및 패딩 (Pure Python/NumPy 버전)
# ==========================================
logger.info("Converting tokens to sequences and padding...")

# 1. word2idx 생성 (Word2Vec 보카 기반)
word2idx = {word: i+2 for i, word in enumerate(w2v_model.wv.index_to_key)}
word2idx['<PAD>'] = 0
word2idx['<UNK>'] = 1

# 2. 토큰 -> 정수 인덱스 변환 함수
def texts_to_sequences(texts, word2idx):
    sequences = []
    for tokens in texts:
        # 단어가 사전에 없으면 <UNK>(1) 할당
        seq = [word2idx.get(token, 1) for token in tokens]
        sequences.append(seq)
    return sequences

# 3. 커스텀 패딩 함수 (Keras의 pad_sequences와 동일하게 동작)
def pad_sequences_manual(sequences, maxlen, padding='post', truncating='post'):
    features = np.zeros((len(sequences), maxlen), dtype=int)
    for i, seq in enumerate(sequences):
        if len(seq) > 0:
            if truncating == 'post':
                truncated = seq[:maxlen]
            else: # 'pre'
                truncated = seq[-maxlen:]
            
            if padding == 'post':
                features[i, :len(truncated)] = truncated
            else: # 'pre'
                features[i, -len(truncated):] = truncated
    return features

# 변환 수행
X_train_seq = texts_to_sequences(X_train, word2idx)
X_val_seq   = texts_to_sequences(X_val, word2idx)
X_test_seq  = texts_to_sequences(X_test, word2idx)

# 패딩 수행 (MAX_LEN 사용)
X_train_pad = pad_sequences_manual(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_val_pad   = pad_sequences_manual(X_val_seq,   maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad  = pad_sequences_manual(X_test_seq,  maxlen=MAX_LEN, padding='post', truncating='post')

logger.info(f"Padding completed. Shape: {X_train_pad.shape}")

2026-01-08 01:44:05,548 - NLP_Project - INFO - Converting tokens to sequences and padding...
2026-01-08 01:44:06,118 - NLP_Project - INFO - Padding completed. Shape: (12042, 358)


정수 인코딩 및 패딩 결과 (X_train_pad)

Shape: (12042, 358)


- 전체 문서를 고정 길이(358)로 통일하여 모델 입력 규격을 맞춤.

- Word2Vec 어휘 사전에 없는 단어는 <UNK>(1)로, 빈 공간은 <PAD>(0)로 처리하여 데이터 손실을 방지하고 연산 효율성을 높임.

In [ ]:
# ==========================================
# Word2Vec 임베딩 행렬 생성
# ==========================================
logger.info("Creating embedding matrix for Word2Vec...")

# 전체 단어 개수 (Padding, Unknown 포함)
vocab_size = len(word2idx)
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

# 사전에 있는 단어들의 벡터를 행렬에 채우기
for word, i in word2idx.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    elif word == '<UNK>':
        # UNK 단어는 랜덤하게 초기화하거나 0으로 둠 (여기선 랜덤 초기화 추천)
        embedding_matrix[i] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))
    # <PAD>는 0번 인덱스이며 이미 0으로 초기화되어 있음

logger.info(f"Embedding matrix created. Shape: {embedding_matrix.shape}")

2026-01-08 01:44:06,126 - NLP_Project - INFO - Creating embedding matrix for Word2Vec...
2026-01-08 01:44:06,202 - NLP_Project - INFO - Embedding matrix created. Shape: (47921, 100)


Word2Vec 임베딩 행렬 결과 (embedding_matrix)

Shape: (47921, 100)

분석:

- Word2Vec으로 학습한 단어들의 의미(100차원 벡터)를 PyTorch Embedding Layer의 가중치로 사용하기 위해 행렬화함.

- 사전에 없는 단어에 대해서는 랜덤 초기화를 적용하여 모델이 학습 과정에서 스스로 의미를 찾아갈 수 있도록 설계함.

### FastText

In [43]:
from gensim.models import FastText

# ==========================================
# 3.4 FastText 모델 학습 (에러 차단 버전)
# ==========================================

logger.info(f"Training FastText model (Skip-gram, Epochs: {epochs})...")

# FastText는 n-gram을 사용하기 때문에 Skip-gram(sg=1)에서 훨씬 강력합니다.
with suppress_stderr():
    ft_model = FastText(
        sentences=X_train,
        vector_size=EMBEDDING_DIM,
        window=WINDOW_SIZE,
        min_count=MIN_COUNT,
        workers=4,
        sg=1, # FastText의 강점을 살리기 위해 Skip-gram 사용
        epochs=epochs
    )

logger.info("FastText training completed.")

# ==========================================
# 3.5 FastText 임베딩 행렬 생성
# ==========================================
logger.info("Creating embedding matrix for FastText...")

# 기존 word2idx(47921개) 규격을 그대로 유지 (비교를 위해)
ft_embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in word2idx.items():
    if word in ft_model.wv:
        ft_embedding_matrix[i] = ft_model.wv[word]
    elif word == '<UNK>':
        ft_embedding_matrix[i] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))

logger.info(f"FastText Embedding matrix created. Shape: {ft_embedding_matrix.shape}")

# 결과 확인 (Word2Vec과 비교해보세요)
for word in ['space', 'computer']:
    if word in ft_model.wv:
        print(f"[FastText - {word}] 유사 단어: {ft_model.wv.most_similar(word, topn=5)}")

2026-01-08 01:44:06,212 - NLP_Project - INFO - Training FastText model (Skip-gram, Epochs: 10)...
2026-01-08 01:45:03,377 - NLP_Project - INFO - FastText training completed.
2026-01-08 01:45:03,400 - NLP_Project - INFO - Creating embedding matrix for FastText...
2026-01-08 01:45:03,809 - NLP_Project - INFO - FastText Embedding matrix created. Shape: (47921, 100)


[FastText - space] 유사 단어: [('deepspace', 0.8876463174819946), ('spacewalk', 0.8690493106842041), ('spaceage', 0.8649821281433105), ('spacehab', 0.8623502254486084), ('spacepac', 0.8566508293151855)]
[FastText - computer] 유사 단어: [('noncomputer', 0.911183774471283), ('compute', 0.9016480445861816), ('computeraided', 0.9013890624046326), ('computercity', 0.8994532823562622), ('computers', 0.8919299840927124)]


FastText 모델 설정: Skip-gram (sg=1), 100차원, 10회 반복 학습.

학습 결과: Word2Vec과 동일한 47,921개의 단어에 대해 n-gram 기반 벡터 생성 완료.

분석 및 비교:

- Word2Vec: 주변 맥락에 의존하여 space-advertising 같은 상업적 연관성이 강하게 나타남.

- FastText: 단어 내부의 부분 문자열을 학습하여 space-spacewalk, computer-computerscience 등 형태와 의미가 직결된 단어들을 훨씬 정확하게 군집화함.

### Glove

In [ ]:
import gensim.downloader as api

# ==========================================
# GloVe 임베딩 로드 및 행렬 생성
# ==========================================
logger.info("Loading GloVe pre-trained model (glove-wiki-gigaword-100)...")

# 100차원짜리 사전 학습된 GloVe 모델 로드
glove_vectors = api.load("glove-wiki-gigaword-100")

logger.info("Creating embedding matrix for GloVe...")
glove_embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, i in word2idx.items():
    if word in glove_vectors:
        glove_embedding_matrix[i] = glove_vectors[word]
    elif word == '<UNK>':
        glove_embedding_matrix[i] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))
    # <PAD>는 0

logger.info(f"GloVe Embedding matrix created. Shape: {glove_embedding_matrix.shape}")

# 결과 확인
for word in ['space', 'computer']:
    if word in glove_vectors:
        print(f"[GloVe - {word}] 유사 단어: {glove_vectors.most_similar(word, topn=5)}")

2026-01-08 01:45:04,163 - NLP_Project - INFO - Loading GloVe pre-trained model (glove-wiki-gigaword-100)...
2026-01-08 01:45:24,840 - NLP_Project - INFO - Creating embedding matrix for GloVe...
2026-01-08 01:45:25,014 - NLP_Project - INFO - GloVe Embedding matrix created. Shape: (47921, 100)


[GloVe - space] 유사 단어: [('nasa', 0.703730583190918), ('spaces', 0.6882482171058655), ('shuttle', 0.6807976365089417), ('earth', 0.672706663608551), ('spacecraft', 0.6626457571983337)]
[GloVe - computer] 유사 단어: [('computers', 0.8751983046531677), ('software', 0.8373122215270996), ('technology', 0.764215886592865), ('pc', 0.7366448640823364), ('hardware', 0.7290390729904175)]


In [ ]:
# ==========================================
# GloVe 임베딩 행렬 생성
# ==========================================
logger.info("Creating embedding matrix for GloVe...")

# vocab_size와 EMBEDDING_DIM은 앞서 정의한 값 사용 (47921, 100)
glove_embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

count_found = 0
for word, i in word2idx.items():
    if word in glove_vectors:
        glove_embedding_matrix[i] = glove_vectors[word]
        count_found += 1
    elif word == '<UNK>':
        # 사전에 없는 단어는 랜덤 초기화
        glove_embedding_matrix[i] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))
    # <PAD>는 0번 인덱스이므로 np.zeros에 의해 이미 0으로 채워져 있음

logger.info(f"GloVe Embedding matrix created. Shape: {glove_embedding_matrix.shape}")
logger.info(f"GloVe Vocab Coverage: {count_found}/{vocab_size-2} ({count_found/(vocab_size-2)*100:.2f}%)")

2026-01-08 01:45:26,189 - NLP_Project - INFO - Creating embedding matrix for GloVe...
2026-01-08 01:45:26,442 - NLP_Project - INFO - GloVe Embedding matrix created. Shape: (47921, 100)
2026-01-08 01:45:26,443 - NLP_Project - INFO - GloVe Vocab Coverage: 35942/47919 (75.01%)


- 인덱스 순서 일치
- 커버리지 확인 (75%)
- 규격 통일 (47921,100)

### 임베딩 모델별 유사도 비교 분석

| 임베딩 방식 | [space] 유사어 | [computer] 유사어 | 분석 및 평가 |
| :--- | :--- | :--- | :--- |
| **Word2Vec** | advertising, nasa, shuttle | shopper, architecture | **도메인 편향적**: 뉴스 데이터 내의 광고/잡지 이름에 휘둘림. |
| **FastText** | spaceage, spacewalk, spacehab | noncomputer, computercity | **형태 중심적**: 단어의 모양(n-gram)을 아주 잘 잡아냄. |
| **GloVe** | nasa, shuttle, earth | software, technology, pc | **범용적/의미론적**: 위키피디아 기반, 사전 학습된 지식 활용 <br>(Coverage: **75.01%**) |

<br>

> * **Word2Vec**은 현재 데이터셋(1990년대 뉴스)의 독특한 맥락을 반영함.
> * **FastText**는 단어의 변형이나 파생어 인식에서 강점을 보임.
> * **GloVe**는 사전 학습된 대규모 데이터를 통해 가장 안정적인 의미 관계를 보존함.

<br>

실험 계획: 동일한 GRU 모델 구조에 위 3가지 행렬을 각각 적용하여,   
"사전 학습된 범용 모델(GloVe)" vs "현재 데이터에 특화된 모델(W2V/FT)" 중 어느 쪽이 분류 성능이 좋은지 비교

# 데이터 로더

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# ==========================================
# Custom Dataset 클래스 정의
# ==========================================
class News20Dataset(Dataset):
    """
    뉴스 데이터셋을 PyTorch 모델이 인식할 수 있는 텐서 형식으로 변환하는 클래스
    """
    def __init__(self, X, y):
        """
        X: 토큰화된 문서 리스트
        y: 라벨 리스트
        """
        # 정수 인덱스(Word Indices)는 Embedding 레이어의 입력값이 되므로 LongTensor(정수형)로 변환합니다.
        self.X = torch.LongTensor(X)
        # 분류 타겟값(Label) 역시 손실 함수 계산을 위해 LongTensor로 처리합니다.
        self.y = torch.LongTensor(y)
        
    def __len__(self):
        """
        데이터셋의 전체 문서 개수를 반환합니다.
        """
        return len(self.X)
    
    def __getitem__(self, idx):
        """
        주어진 인덱스(idx)에 해당하는 문서와 라벨을 반환합니다.
        """
        return self.X[idx], self.y[idx]

# ==========================================
# DataLoader 생성 (배치 관리자)
# ==========================================
BATCH_SIZE = 64 # 64~128 사이는 텍스트 분류에서 가장 범용적으로 쓰입니다.

# Train, Val, Test 각각의 로더 생성
train_loader = DataLoader(News20Dataset(X_train_pad, y_train), batch_size=BATCH_SIZE, shuffle=True) # shuffle=True로 학습 데이터를 무작위로 섞어서 과적합 방지
val_loader   = DataLoader(News20Dataset(X_val_pad, y_val), batch_size=BATCH_SIZE, shuffle=False) # shuffle=False로 검증 데이터는 무작위로 섞지 않습니다.
test_loader  = DataLoader(News20Dataset(X_test_pad, y_test), batch_size=BATCH_SIZE, shuffle=False) # shuffle=False로 테스트 데이터는 무작위로 섞지 않습니다.

logger.info(f"DataLoader 준비 완료. Train batches: {len(train_loader)}")

2026-01-08 01:45:26,513 - NLP_Project - INFO - DataLoader 준비 완료. Train batches: 189


- 배치 사이즈(64): 하드웨어 리소스와 학습 속도를 모두 고려한 수치로 설정

- 학습 단위(189 Batches): 1 에포크 내에서 총 189회의 Step을 거치며 모델이 정교하게 가중치를 조정하도록 구성

- 인덱싱 최적화: 모든 단어 데이터를 LongTensor로 변환하여 임베딩 층의 메모리 참조 오류를 방지하고 연산 속도를 확보

# 예측 모델 설계 (GRU) 및 임베딩 별 학습 진행

본 프로젝트에서는 텍스트 분류를 위한 메인 모델로 GRU(Gated Recurrent Unit) 기반의 RNN 구조를 채택하였습니다.   
선택의 주요 근거는 다음과 같습니다.

1. 연산 효율성 및 학습 속도

    - 파라미터 최적화: GRU는 LSTM의 3개 게이트(Input, Forget, Output)를 2개(Update, Reset)로 간소화하여 파라미터 수가 약 25~30% 적습니다.

    - 로컬 환경 최적화: Apple Silicon(MPS)을 사용하는 제한된 로컬 리소스 환경에서, LSTM 대비 빠른 연산 속도를 제공하여 임베딩 방식별 반복 실험을 효율적으로 수행할 수 있습니다.

2. 데이터셋 규모에 적합한 구조

    - 과적합(Overfitting) 방지: 본 미션에서 사용하는 NewsGroup 20 데이터셋(약 1.8만 개)은 딥러닝 기준에서 대규모는 아닙니다. 구조가 단순한 GRU는 복잡한 LSTM보다 적은 데이터셋에서 더 우수한 일반화 성능을 보이는 경향이 있습니다.

3. 실험 설계의 일관성

    - 변수 통제: 본 실험의 핵심 목표는 Word2Vec, FastText, GloVe 임베딩 방식에 따른 성능 차이를 관찰하는 것입니다. 따라서 분류 모델의 구조를 GRU로 고정함으로써, 성능 변화의 원인이 모델 구조가 아닌 '임베딩 방식'에 있음을 명확히 규명하고자 합니다.

### 하이퍼파리미터 설정

In [91]:
VOCAB_SIZE = len(word2idx) # 사전 크기 (단어 총 개수)
HIDDEN_DIM = 128    # GRU 은닉층 크기
N_LAYERS = 2        # 레이어를 2층으로 쌓아 복잡한 문맥 파악
DROPOUT = 0.5       # 과적합 방지를 위해 학습 시 뉴런의 50%를 무작위로 끔
OUTPUT_DIM = 20     # 뉴스 카테고리 개수 (고정)
LEARNING_RATE = 0.001
INPUT_DIM = EMBEDDING_DIM # 위에서 정의한 EMBEDDING_DIM은 여기서도 입력 차원으로 자동 활용됩니다.

### 모델 정의

In [ ]:
import torch.nn as nn

# ==========================================
# GRU 모델 클래스 정의
# ==========================================
class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, embedding_matrix):
        super(GRUClassifier, self).__init__()
        
        # 1. 임베딩 레이어: 준비한 행렬(W2V/FT/GloVe)을 주입
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = True # 미세 조정(Fine-tuning) 허용
        
        # 2. GRU 레이어: 양방향 설정
        self.gru = nn.GRU(embedding_dim, 
                          hidden_dim, 
                          num_layers=n_layers, 
                          batch_first=True, 
                          dropout=dropout if n_layers > 1 else 0, 
                          bidirectional=True) # 양방향 GRU
        
        # 3. 출력 레이어: 양방향 출력이므로 입력 차원은 hidden_dim * 2
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: [batch_size, seq_len]
        embedded = self.dropout(self.embedding(x))
        
        # output: [batch_size, seq_len, hid_dim * 2]
        # hidden: [n_layers * 2, batch_size, hid_dim]
        output, hidden = self.gru(embedded)
        
        # hidden[-2,:,:]: 정방향 GRU의 마지막 층 hidden state
        # hidden[-1,:,:]: 역방향 GRU의 마지막 층 hidden state
        # 두 벡터를 가로로 이어 붙여(concatenate) 마지막 층의 정방향과 역방향 hidden state를 합침
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        
        return self.fc(self.dropout(hidden))

logger.info("GRU Classifier class defined.")

2026-01-08 01:45:26,541 - NLP_Project - INFO - GRU Classifier class defined.


### 학습 설계

In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tqdm.auto import tqdm # 진척도 바를 위한 라이브러리

# 1. 아웃풋 폴더 생성
output_dirs = ['../results/weights', '../results/logs', '../results/plots']
for d in output_dirs:
    os.makedirs(d, exist_ok=True)

# 2. 학습 엔진
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs, model_name):
    """
    모델 학습 및 검증을 수행하는 핵심 함수입니다.
    
    Args:
        model (nn.Module): 학습할 PyTorch 모델
        train_loader (DataLoader): 학습 데이터 로더
        val_loader (DataLoader): 검증 데이터 로더
        optimizer (Optimizer): 최적화 알고리즘 (예: Adam)
        criterion (Loss): 손실 함수 (예: CrossEntropyLoss)
        epochs (int): 총 학습 반복 횟수
        model_name (str): 결과 파일 저장을 위한 실험명
        
    Returns:
        history (dict): 학습 과정의 Loss와 Accuracy 기록
    """
    best_val_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    log_file_path = f"../results/logs/{model_name}_train_log.txt"
    
    with open(log_file_path, "w") as f:
        f.write(f"Starting Training: {model_name}\n")
        
        for epoch in range(epochs):
            # --- TRAIN ---
            model.train()
            train_loss, train_correct = 0, 0
            
            # tqdm으로 감싸서 진척도 바 출력
            train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
            for texts, labels in train_pbar:
                texts, labels = texts.to(device), labels.to(device)

                optimizer.zero_grad()           # 1. 기울기 초기화
                outputs = model(texts)          # 2. 순전파 (Forward)
                loss = criterion(outputs, labels) # 3. 손실 계산
                loss.backward()                 # 4. 역전파 (Backward)
                optimizer.step()                # 5. 가중치 업데이트
                
                train_loss += loss.item()
                train_correct += (outputs.argmax(1) == labels).sum().item()
                
                # 실시간으로 pbar에 Loss 표시
                train_pbar.set_postfix({'loss': f"{loss.item():.4f}"})
            
            # --- VALIDATION ---
            model.eval()
            val_loss, val_correct = 0, 0
            all_preds, all_labels = [], []
            
            val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
            with torch.no_grad(): # 검증 시에는 기울기 계산 비활성화 (메모리 절약)
                for texts, labels in val_pbar:
                    texts, labels = texts.to(device), labels.to(device)
                    outputs = model(texts)
                    v_loss = criterion(outputs, labels)
                    val_loss += v_loss.item()
                    
                    preds = outputs.argmax(1)
                    val_correct += (preds == labels).sum().item()

                    # 최종 리포트 작성을 위해 예측값 수집
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
            
            # --- 지표 계산 및 로깅 ---
            t_loss = train_loss / len(train_loader)
            v_loss = val_loss / len(val_loader)
            t_acc = train_correct / len(train_loader.dataset)
            v_acc = val_correct / len(val_loader.dataset)
            
            history['train_loss'].append(t_loss); history['val_loss'].append(v_loss)
            history['train_acc'].append(t_acc); history['val_acc'].append(v_acc)
            
            status = f"Epoch {epoch+1:02d} | Train Loss: {t_loss:.4f} | Val Acc: {v_acc:.4f}"
            print(f"\n=> {status}") # tqdm이랑 안 겹치게 줄바꿈 추가
            f.write(status + "\n")
            
            # [Checkpoint] 검증 정확도 최고 기록 갱신 시 모델 저장
            if v_acc > best_val_acc:
                best_val_acc = v_acc
                torch.save(model.state_dict(), f"../results/weights/best_{model_name}.pt")
        
        # 학습 종료 후 최종 분류 리포트(Precision, Recall, F1) 저장
        f.write("\nFinal Classification Report:\n" + classification_report(all_labels, all_preds))
        
    return history

# 3. 결과 시각화 함수
def plot_results(history, model_name):
    """
    학습 결과(Loss, Accuracy)를 인터랙티브 그래프로 시각화하고 HTML로 저장합니다.
    """
    epochs = list(range(1, len(history['train_loss']) + 1))
    
    # 서브플롯 생성 (1행 2열)
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f'{model_name} Loss', f'{model_name} Accuracy'))
    
    # 1. Loss 그래프
    fig.add_trace(go.Scatter(x=epochs, y=history['train_loss'], name='Train Loss', mode='lines+markers'), row=1, col=1)
    fig.add_trace(go.Scatter(x=epochs, y=history['val_loss'], name='Val Loss', mode='lines+markers'), row=1, col=1)
    
    # 2. Accuracy 그래프
    fig.add_trace(go.Scatter(x=epochs, y=history['train_acc'], name='Train Acc', mode='lines+markers'), row=1, col=2)
    fig.add_trace(go.Scatter(x=epochs, y=history['val_acc'], name='Val Acc', mode='lines+markers'), row=1, col=2)
    
    fig.update_layout(title_text=f"Experiment Result: {model_name}", height=500, showlegend=True)
    
    # HTML 저장 및 출력
    fig.write_html(f"../results/plots/{model_name}_interactive.html")
    fig.show()

# 4. 실험 관리자 (이걸로 호출)
def run_experiment(matrix, name, epochs=10):
    """
    특정 임베딩 행렬을 적용하여 전체 실험 프로세스(생성-학습-저장-시각화)를 수행합니다.
    
    Args:
        matrix (numpy.ndarray): 사전 학습된 임베딩 가중치 행렬
        name (str): 실험 식별자 (예: 'W2V_GRU')
        epochs (int): 학습 에폭 수
    """
    logger.info(f">>> Starting Experiment: {name}")
    # 매번 모델을 새로 생성 (가중치 초기화 보장)
    model = GRUClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, DROPOUT, matrix).to(device)
    
    # 최적화 도구 및 손실 함수 재설정
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    # 학습 엔진 가동
    history = train_model(model, train_loader, val_loader, optimizer, criterion, epochs, name)
    
    # 학습 결과 시각화
    plot_results(history, name)
    
    return history

**기술적 포인트 및 설계 전략**

| 컴포넌트 | 핵심 설정 및 기술 | 분석 및 기대 효과 |
| :--- | :--- | :--- |
| **Embedding Layer** | **Fine-tuning 허용**<br>(`requires_grad=True`) | 사전 학습된 범용 벡터를 초기값으로 하되, 우리 뉴스 데이터의 특성에 맞춰 미세 조정하여 **도메인 적응력**을 극대화함. |
| **GRU Structure** | **Bidirectional**<br>(양방향) | 문장을 정방향과 역방향으로 동시에 읽어, 문맥의 앞뒤 정보를 모두 포착하여 **분류 정확도**를 높임. |
| **Output Layer** | **Concat Hidden**<br>(`dim=1`) | 양방향 GRU의 마지막 은닉 상태(Last Hidden States)를 결합하여 문장의 **핵심 요약 정보**를 압축적으로 전달함. |
| **Train Engine** | **Checkpointing**<br>(`best_val_acc`) | 검증 정확도가 갱신될 때마다 모델을 저장하여, 과적합 이전의 **최적 성능 모델**을 확보함. |
| **Monitoring** | **Automated Logging**<br>(`.txt`, `.html`) | 모든 학습 로그와 시각화 결과를 파일로 자동 저장하여, 실험 후에도 **결과 복기 및 보고서 작성**이 용이함. |
| **Architecture** | **Modular Design**<br>(`run_experiment`) | 임베딩 행렬만 인자로 받아 실험을 수행하는 함수형 구조로, 코드 중복을 제거하고 **실험의 확장성**을 확보함. |

<br>

**학습 엔진(Train Engine) 설계 의도**
> * **실험의 관리와 재현성**에 초점을 맞춘 MLOps 기초 단계를 구현함.
> * `tqdm`을 통한 실시간 모니터링과 `Plotly`를 통한 인터랙티브 결과 확인

### 학습 1. Word2Vec x GRU

In [ ]:
# 1. Word2Vec
history_w2v = run_experiment(embedding_matrix, "W2V_GRU", epochs=10)

2026-01-08 01:45:26,588 - NLP_Project - INFO - >>> Starting Experiment: W2V_GRU


Epoch 1/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

Epoch 1/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 01 | Train Loss: 2.2518 | Val Acc: 0.4337


Epoch 2/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

...

Epoch 9/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 09 | Train Loss: 0.2620 | Val Acc: 0.8084


Epoch 10/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

Epoch 10/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 10 | Train Loss: 0.2009 | Val Acc: 0.8376


### 학습 2. FastText x GRU

In [ ]:
# 2. FastText
history_ft = run_experiment(ft_embedding_matrix, "FT_GRU", epochs=10)

2026-01-08 03:02:04,673 - NLP_Project - INFO - >>> Starting Experiment: FT_GRU


Epoch 1/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

Epoch 1/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 01 | Train Loss: 2.3027 | Val Acc: 0.4716


Epoch 2/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

...

Epoch 9/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 09 | Train Loss: 0.2975 | Val Acc: 0.8396


Epoch 10/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

Epoch 10/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 10 | Train Loss: 0.2537 | Val Acc: 0.8406


### 학습 3. Glove x GRU

In [ ]:
# 3. GloVe
history_glove = run_experiment(glove_embedding_matrix, "GloVe_GRU", epochs=10)

2026-01-08 03:29:16,747 - NLP_Project - INFO - >>> Starting Experiment: GloVe_GRU


Epoch 1/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

Epoch 1/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 01 | Train Loss: 2.3932 | Val Acc: 0.3839


Epoch 2/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

...

Epoch 9/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 09 | Train Loss: 0.3274 | Val Acc: 0.8240


Epoch 10/10 [Train]:   0%|          | 0/189 [00:00<?, ?it/s]

Epoch 10/10 [Val]:   0%|          | 0/48 [00:00<?, ?it/s]


=> Epoch 10 | Train Loss: 0.2825 | Val Acc: 0.8310


# 결과 분석 및 평가

### 평가 지표 분석

In [75]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate_best_models_comprehensive(experiments, test_loader):
    """
    저장된 베스트 모델들을 불러와 테스트 데이터셋에서 
    Accuracy, Precision, Recall, F1-score(Macro/Weighted)를 종합 산출합니다.
    """
    final_summary = []
    
    for exp in experiments:
        name = exp['name']
        matrix = exp['matrix']
        # 저장된 파일명 규칙에 맞게 경로 설정 (훈련 시 저장한 이름과 일치해야 함)
        weight_path = f"../results/weights/best_{name}_GRU.pt" 
        
        # 1. 모델 복원
        model = GRUClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, DROPOUT, matrix).to(device)
        model.load_state_dict(torch.load(weight_path))
        model.eval()
        
        all_preds = []
        all_labels = []
        
        # 2. 예측 수행
        with torch.no_grad():
            for texts, labels in test_loader:
                texts, labels = texts.to(device), labels.to(device)
                outputs = model(texts)
                preds = outputs.argmax(1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        # 3. 종합 지표 산출
        # Macro: 모든 클래스를 동일한 비중으로 평균 (소수 클래스 성능 확인에 유리)
        # Weighted: 클래스별 샘플 수에 비중을 두어 평균
        acc = accuracy_score(all_labels, all_preds)
        p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        pw, rw, f1w, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        
        final_summary.append({
            "Model": name,
            "Accuracy": acc,
            "Macro Precision": p,
            "Macro Recall": r,
            "Macro F1": f1,
            "Weighted F1": f1w
        })
        
        logger.info(f"Model: {name} 평가 완료 | Test Acc: {acc:.4f} | Macro F1: {f1:.4f}")
    
    # 데이터프레임으로 변환하여 보기 좋게 출력
    df_results = pd.DataFrame(final_summary).set_index("Model")
    return df_results

# 실행
comparison_df = evaluate_best_models_comprehensive([
    {"name": "W2V", "matrix": embedding_matrix},
    {"name": "FT", "matrix": ft_embedding_matrix},
    {"name": "GloVe", "matrix": glove_embedding_matrix}
], test_loader)

# 최종 결과 표 출력
display(comparison_df.sort_values(by="Macro F1", ascending=False))

2026-01-08 15:22:52,006 - NLP_Project - INFO - Model: W2V 평가 완료 | Test Acc: 0.8374 | Macro F1: 0.8344
2026-01-08 15:23:04,055 - NLP_Project - INFO - Model: FT 평가 완료 | Test Acc: 0.8440 | Macro F1: 0.8395
2026-01-08 15:23:17,658 - NLP_Project - INFO - Model: GloVe 평가 완료 | Test Acc: 0.8353 | Macro F1: 0.8318


,Accuracy,Macro Precision,Macro Recall,Macro F1,Weighted F1
Model,,,,,
FT,0.844049,0.842003,0.840436,0.839475,0.843989
W2V,0.837407,0.837907,0.834414,0.834447,0.837627
GloVe,0.835282,0.840412,0.829634,0.831782,0.835447


[핵심 총평] 
> 실험 결과, 세 가지 임베딩 모델 모두 83~84% 범위의 박빙의 성능을 기록하였습니다.   
> 이는 본 실험에서 사용된 20 Newsgroups 데이터의 Train/Val/Test 세트가 동일한 시계열과 문체(90년대 뉴스)를 공유하고 있어,   
> 양방향 GRU 모델이 임베딩의 종류와 관계없이 카테고리별 고유 패턴을 안정적으로 학습했음을 의미합니다.   
> 즉, 특정 임베딩의 우월성보다는 텍스트 분류 작업 자체에 대한 모델의 높은 적합성을 먼저 확인할 수 있었습니다.

[개별 모델의 성능]
> 단, 미미한 차이 속에서도 각 임베딩 알고리즘의 설계 철학에 따른 특징이 결과에 반영되었을 것이라고 가설을 제시합니다.

1. FastText: 본 프로젝트에서 FastText가 모든 지표에서 가장 우수한 성능을 보였습니다. 이는 뉴스 데이터 특성상 발생하는 오타나 다양한 파생어들을 FastText의 Subword(n-gram) 정보 활용 능력이 효과적으로 처리했음을 의미.

2. Word2Vec: 현재 뉴스 데이터셋 내부의 문맥을 직접 학습한 Word2Vec은 FastText와 근소한 차이로 2위를 기록했습니다. 도메인 특화된 말뭉치 학습이 분류 성능에 긍정적인 영향을 미쳤습니다.

3. GloVe: 위키피디아 등 대규모 범용 데이터로 학습된 GloVe는 90년대 뉴스 데이터라는 특정 도메인 맥락에서 W2V나 FT보다 다소 낮은 적응력을 보였습니다.

### 카테고리별 성능 격차 분석
- 모델별로 어떤 주제를 잘 맞추고 어떤 주제에서 고전했는지 테스트 데이터를 통해 직접 산출합니다.

In [89]:
import pandas as pd
from sklearn.metrics import classification_report

def analyze_test_top_bottom(results, target_names):
    """
    최종 테스트 결과를 바탕으로 모델별 상/하위 카테고리를 분석합니다.
    """
    for res in results:
        model_name = res['Model']
        preds = res['preds']
        labels = res['labels']
        
        # 1. 클래스별 상세 리포트 생성 (dict 형태)
        report = classification_report(labels, preds, target_names=target_names, output_dict=True)
        
        # 2. 개별 클래스 지표만 추출 (accuracy, macro avg 등 제외)
        class_metrics = []
        for name in target_names:
            if name in report:
                class_metrics.append({
                    "Category": name,
                    "F1-Score": report[name]['f1-score']
                })
        
        df_metrics = pd.DataFrame(class_metrics)
        
        # 3. 출력
        print(f"[{model_name} Test Results Summary]")
        print(f"Overall Accuracy: {report['accuracy']:.4f}")
        print("-" * 50)
        
        top3 = df_metrics.nlargest(3, 'F1-Score')
        print("Top 3 Categories (F1-Score):")
        for _, row in top3.iterrows():
            print(f" - {row['Category']}: {row['F1-Score']:.4f}")
            
        print("\nBottom 3 Categories (F1-Score):")
        bottom3 = df_metrics.nsmallest(3, 'F1-Score')
        for _, row in bottom3.iterrows():
            print(f" - {row['Category']}: {row['F1-Score']:.4f}")
        print("\n" + "="*50 + "\n")

# 실행 (Step 8에서 얻은 results 리스트 활용)
analyze_test_top_bottom(results, target_names)

[W2V Test Results Summary]
Overall Accuracy: 0.8374
--------------------------------------------------
Top 3 Categories (F1-Score):
 - rec.sport.hockey: 0.9340
 - rec.motorcycles: 0.9235
 - sci.crypt: 0.9156

Bottom 3 Categories (F1-Score):
 - comp.sys.ibm.pc.hardware: 0.6944
 - talk.religion.misc: 0.7045
 - comp.sys.mac.hardware: 0.7139


[FT Test Results Summary]
Overall Accuracy: 0.8440
--------------------------------------------------
Top 3 Categories (F1-Score):
 - rec.sport.hockey: 0.9421
 - sci.crypt: 0.9275
 - rec.sport.baseball: 0.9227

Bottom 3 Categories (F1-Score):
 - talk.religion.misc: 0.6831
 - comp.sys.ibm.pc.hardware: 0.7183
 - comp.graphics: 0.7443


[GloVe Test Results Summary]
Overall Accuracy: 0.8353
--------------------------------------------------
Top 3 Categories (F1-Score):
 - rec.sport.hockey: 0.9377
 - sci.med: 0.9365
 - rec.sport.baseball: 0.9263

Bottom 3 Categories (F1-Score):
 - comp.sys.ibm.pc.hardware: 0.6543
 - talk.religion.misc: 0.6730
 - sci.elect

1. 독보적 강점: 스포츠 및 전문 과학

- rec.sport.hockey, rec.sport.baseball, sci.crypt 등은 모든 모델에서 0.9점 이상의 고득점을 기록했습니다.

- 이유: 해당 분야는 'puck', 'encryption', 'homerun' 등 타 분야와 중복되지 않는 **고유 어휘(Domain-Specific Words)**가 뚜렷하여 임베딩 공간에서 분류가 매우 명확하게 일어납니다.

2. 공통적 난제: 하드웨어 및 종교 간 간섭

- 모든 모델에서 comp.sys.ibm.pc.hardware, comp.sys.mac.hardware, talk.religion.misc가 최하위권에 머물고 있습니다.

- 하드웨어: 'disk', 'scsi', 'monitor' 등 IBM PC와 Mac에서 공통으로 쓰이는 단어가 너무 많아 모델이 두 클래스의 미세한 차이를 구분하지 못합니다.

- 종교: 'god', 'bible', 'moral' 등의 단어가 기독교(soc.religion.christian)와 일반 종교(talk.religion.misc) 섹션에 산재해 있어 문맥적 혼선이 발생합니다.

3. 임베딩 모델별 특성 발견

- FastText: sci.crypt 등 전문 용어에서 강세를 보입니다. n-gram 특성상 복잡한 기술 용어나 오타가 섞인 단어 대응력이 좋기 때문으로 풀이됩니다.

- GloVe: sci.med(의학)에서 0.93으로 매우 높은 점수를 보였습니다. 이는 대규모 위키피디아로 학습된 GloVe의 사전 지식이 의학적 상식 키워드와 잘 매칭되었음을 시사합니다.

### 모델 예측 결과 샘플 분석 (Qualitative Analysis)

앞서 확인한 정량적 지표(Accuracy, F1-Score)를 바탕으로, 각 모델이 실제 텍스트를 어떻게 카테고라이징하는지 샘플 데이터를 통해 정성적으로 확인합니다. 통합 분석 함수인 run_analysis를 활용하여 모델의 판단 근거를 직접 검토합니다.

**분석 도구 사용법**

제공된 통합 함수는 세 가지 모드를 통해 모델의 예측 결과를 다각도로 분석할 수 있도록 설계되었습니다.

- mode='random': 전체 테스트 데이터 중 무작위 샘플 추출 (일반적인 성능 체감용)

- mode='category': 특정 주제를 지정하여 집중 분석 (취약 카테고리 분석용)

- mode='all': 20개 전체 카테고리에서 1개씩 샘플 추출 (전수 검사 및 보고용)

**분석의 의의 및 관점**
 
수치적인 지표(정량 평가)가 주지 못하는 모델의 판단 논리를 이해하는 데 목적이 있습니다.

1. 전처리 품질 검수: 복원된 텍스트(Text Snippet)에 핵심 키워드가 잘 살아있는지 확인하여 전처리 파이프라인의 적절성을 최종 점검합니다.

2. 오답 패턴 파악 (Error Analysis): 하이라이트된 오답 셀을 분석하여, 모델이 어떤 단어에 현혹되어 잘못된 예측을 내렸는지 정성적으로 분석합니다.

3. 임베딩 모델별 개성 확인: 동일한 문장에 대해 W2V, FT, GloVe가 서로 다른 예측을 내놓는 경우, 각 임베딩이 강조하는 문맥의 차이를 이해할 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np
import torch
from IPython.display import display

# 0. 사전 준비 (idx -> word 변환)
idx2word = {v: k for k, v in word2idx.items()}

def run_analysis(mode='random', target=None, num_samples=5):
    """
    통합 모델 분석 도구: 랜덤 추출, 특정 카테고리 분석, 전체 훑어보기 기능을 제공합니다.
    
    Args:
        mode (str): 'random' (무작위), 'category' (특정 주제), 'all' (주제별 1개씩)
        target (int or str): 'category' 모드일 때 선택할 주제 (인덱스 번호 또는 이름)
        num_samples (int): 추출할 샘플 개수
    """
    # 전역 변수 사용 (model_configs, X_test_pad, y_test, target_names)
    # 실제 사용 시엔 인자로 받아도 되지만, 노트북 환경 편의성을 위해 전역 참조 허용
    
    y_test_arr = np.array(y_test)
    sample_indices = []

    # --- [Step 1] 타겟 카테고리 처리 (이름 몰라도 번호로 가능하게) ---
    selected_category_name = None
    
    if mode == 'category':
        # 타겟 입력이 없거나 틀렸을 때 -> 메뉴판 출력
        if target is None:
            print("❌ [Error] 분석할 카테고리가 지정되지 않았습니다.")
            print("👇 아래 목록에서 번호(Index)나 이름(Name)을 골라 'target' 인자에 넣어주세요.\n")
            for i, name in enumerate(target_names):
                print(f"  [{i:>2}] {name}")
            return None
        
        # 정수(인덱스) 입력 처리
        if isinstance(target, int):
            if 0 <= target < len(target_names):
                selected_category_name = target_names[target]
            else:
                print(f"❌ [Error] 유효하지 않은 인덱스입니다. (0 ~ {len(target_names)-1} 사이)")
                return None
        # 문자열(이름) 입력 처리
        elif isinstance(target, str):
            if target in target_names:
                selected_category_name = target
            else:
                print(f"❌ [Error] '{target}'는 존재하지 않는 카테고리입니다.")
                return None
                
        print(f"🔎 ['{selected_category_name}'] 카테고리를 집중 분석합니다...")

    # --- [Step 2] 샘플링 로직 ---
    if mode == 'all':
        print("🔎 모든 카테고리에서 샘플을 하나씩 추출합니다...")
        for class_idx in range(len(target_names)):
            idx = np.where(y_test_arr == class_idx)[0]
            if len(idx) > 0: sample_indices.append(idx[0])
            
    elif mode == 'random':
        print(f"🎲 전체 데이터 중 무작위로 {num_samples}개를 추출합니다...")
        sample_indices = np.random.choice(len(y_test_arr), num_samples, replace=False)
        
    elif mode == 'category':
        class_idx = target_names.index(selected_category_name)
        idx = np.where(y_test_arr == class_idx)[0]
        if len(idx) == 0:
            print("⚠️ 해당 카테고리의 테스트 데이터가 없습니다.")
            return None
        sample_indices = np.random.choice(idx, min(num_samples, len(idx)), replace=False)

    # --- [Step 3] 데이터 복원 및 예측 ---
    results = []
    input_tensor = torch.LongTensor(X_test_pad[sample_indices]).to(device)
    
    # 원문 복원
    for i, s_idx in enumerate(sample_indices):
        raw_tokens = X_test_pad[s_idx]
        decoded = " ".join([idx2word.get(int(t), "?") for t in raw_tokens if t != 0])
        results.append({
            "Category": target_names[y_test_arr[s_idx]],
            "Text Snippet": decoded[:80] + "...", # 가독성을 위해 길이 조절
            "Ground Truth": target_names[y_test_arr[s_idx]]
        })

    # 모델별 예측 (가중치 로드 -> 예측 -> 메모리 해제)
    for config in model_configs:
        model = GRUClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, 
                              OUTPUT_DIM, N_LAYERS, DROPOUT, config['matrix']).to(device)
        model.load_state_dict(torch.load(config['weight']))
        model.eval()
        
        with torch.no_grad():
            outputs = model(input_tensor)
            preds = outputs.argmax(1).cpu().numpy()
            for i, p_idx in enumerate(preds):
                results[i][config['name']] = target_names[p_idx]
        del model

    df = pd.DataFrame(results)

    # --- [Step 4] 스타일링 및 출력 (내장) ---
    styled = df.style.set_properties(**{
        'text-align': 'left', 'color': 'black', 'background-color': '#ffffff',
        'border': '1px solid #eeeeee'
    }).set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#f8f9fa'), ('color', '#333'), ('font-weight', 'bold')]}
    ])

    def apply_highlight(col):
        if col.name in ['Word2Vec', 'FastText', 'GloVe']:
            return [
                'background-color: #ffd7d7; color: #d93025; font-weight: bold' if v != df.loc[i, 'Ground Truth'] 
                else 'background-color: #e6f4ea; color: #137333;' 
                for i, v in col.items()
            ]
        return ['background-color: #ffffff'] * len(col)

    # 스타일 적용된 객체 반환
    return styled.apply(apply_highlight, axis=0)

In [88]:
# 모든 카테고리(0~19번)에서 첫 번째 샘플을 하나씩 추출하여 비교
run_analysis(mode='all')

🔎 모든 카테고리에서 샘플을 하나씩 추출합니다...


,Category,Text Snippet,Ground Truth,Word2Vec,FastText,GloVe
0,alt.atheism,article naren writes list killings name religion iraniraq war civil war su...,alt.atheism,alt.atheism,alt.atheism,alt.atheism
1,comp.graphics,toronto siggraph chances art graphics animation indigo ken evans inc...,comp.graphics,comp.graphics,comp.graphics,comp.graphics
2,comp.os.ms-windows.misc,article warren iii gerry swetsky writes set shortcut key return program ma...,comp.os.ms-windows.misc,comp.os.ms-windows.misc,comp.os.ms-windows.misc,comp.sys.mac.hardware
3,comp.sys.ibm.pc.hardware,anyone help would greatly appreciate christmas built computer used parts d...,comp.sys.ibm.pc.hardware,comp.sys.ibm.pc.hardware,comp.sys.ibm.pc.hardware,comp.sys.ibm.pc.hardware
4,comp.sys.mac.hardware,hello remember running across back years ago nubus board umpteen simm slot...,comp.sys.mac.hardware,comp.sys.mac.hardware,comp.sys.mac.hardware,comp.sys.mac.hardware
5,comp.windows.x,old sun gets occasional use started console messages startup fully started...,comp.windows.x,comp.windows.x,comp.windows.x,comp.windows.x
6,misc.forsale,ive got sets sample cds sale set one towards rap drum loops drum sou...,misc.forsale,misc.forsale,misc.forsale,misc.forsale
7,rec.autos,wharfie writes compare either porsche tell designed right dollar cars driven fas...,rec.autos,rec.autos,rec.autos,rec.autos
8,rec.motorcycles,one little thing last year cop pulled centigrade started snowing right liv...,rec.motorcycles,rec.autos,rec.motorcycles,rec.motorcycles
9,rec.sport.baseball,last night boston red sox win games games beating seattle roger clemson pitch do...,rec.sport.baseball,rec.sport.baseball,rec.sport.baseball,rec.sport.baseball


In [84]:
run_analysis(mode='random', num_samples=5)

🎲 전체 데이터 중 무작위로 5개를 추출합니다...


,Category,Text Snippet,Ground Truth,Word2Vec,FastText,GloVe
0,comp.windows.x,article zvi writes author wcl current care taker name found dist tree trie...,comp.windows.x,comp.windows.x,talk.religion.misc,alt.atheism
1,soc.religion.christian,despite trendy liberal feminist tendencies fact basically agree saying rebut nan...,soc.religion.christian,soc.religion.christian,talk.religion.misc,soc.religion.christian
2,sci.med,article john odonnell writes imho lyme disease sent private email summary ...,sci.med,sci.med,sci.med,sci.med
3,sci.med,article lin writes first question bad xray ive heard nothing compared amount tim...,sci.med,sci.med,sci.med,sci.med
4,rec.sport.baseball,jayson stark thats fits perfectly category anyone writes dean palmer homer...,rec.sport.baseball,rec.sport.baseball,rec.sport.baseball,rec.sport.baseball


In [ ]:
# 번호로 불러오기
run_analysis(mode='category', target=10, num_samples=5)

🔎 ['rec.sport.hockey'] 카테고리를 집중 분석합니다...


,Category,Text Snippet,Ground Truth,Word2Vec,FastText,GloVe
0,rec.sport.hockey,article richard casares writes last time saw hockey league inner city well actua...,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey
1,rec.sport.hockey,article lord vader writes sure asked times wondered since heard hell nickname ha...,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey
2,rec.sport.hockey,article ralph writes article mathew writes penguins get patrick win ...,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey
3,rec.sport.hockey,hey man spent past season learning skate played couple sessions mock hockey read...,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey
4,rec.sport.hockey,world championships germany group results sweden canada geoff sanderson kevin di...,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey,rec.sport.hockey


In [ ]:
# 이름으로 넣기
run_analysis(mode='category', target='talk.religion.misc', num_samples=5)

🔎 ['talk.religion.misc'] 카테고리를 집중 분석합니다...


,Category,Text Snippet,Ground Truth,Word2Vec,FastText,GloVe
0,talk.religion.misc,jim andy incorrect believe facts processes exist present physical evidence proce...,talk.religion.misc,alt.atheism,alt.atheism,alt.atheism
1,talk.religion.misc,article pegasus writes article joshua wrote article writes let...,talk.religion.misc,talk.religion.misc,talk.religion.misc,talk.religion.misc
2,talk.religion.misc,yawn church kibology first better...,talk.religion.misc,soc.religion.christian,soc.religion.christian,soc.religion.christian
3,talk.religion.misc,article stephen writes article writes name stephen writes think david koresh did...,talk.religion.misc,talk.religion.misc,talk.religion.misc,talk.religion.misc
4,talk.religion.misc,groups continue believe christians worship sabbath saturday bestknown seventhday...,talk.religion.misc,soc.religion.christian,soc.religion.christian,soc.religion.christian


In [ ]:
# target 안 넣으면 목록 출력 (노트북에서 보여주는 것 고려하여, input으로 받는 건 미구현)
run_analysis(mode='category')

❌ [Error] 분석할 카테고리가 지정되지 않았습니다.
👇 아래 목록에서 번호(Index)나 이름(Name)을 골라 'target' 인자에 넣어주세요.

  [ 0] alt.atheism
  [ 1] comp.graphics
  [ 2] comp.os.ms-windows.misc
  [ 3] comp.sys.ibm.pc.hardware
  [ 4] comp.sys.mac.hardware
  [ 5] comp.windows.x
  [ 6] misc.forsale
  [ 7] rec.autos
  [ 8] rec.motorcycles
  [ 9] rec.sport.baseball
  [10] rec.sport.hockey
  [11] sci.crypt
  [12] sci.electronics
  [13] sci.med
  [14] sci.space
  [15] soc.religion.christian
  [16] talk.politics.guns
  [17] talk.politics.mideast
  [18] talk.politics.misc
  [19] talk.religion.misc


### 향후 모델 개선 방향

1. 클래스 불균형 및 간섭 해결 (Data Centric)

- 오답 위주 샘플링: 현재 하위권인 '하드웨어'와 '종교' 데이터만 따로 모아 추가 학습(Fine-tuning)을 진행하거나, 두 클래스 간의 차이를 보여주는 키워드를 불용어 리스트에서 제외하여 변별력을 높여야 합니다.

2. Attention 메커니즘 도입 (Architecture)

- 현재 GRU는 문장 전체를 압축하지만, 모든 단어를 동일 비중으로 처리합니다. Attention 레이어를 추가하여 분류에 결정적인 힌트가 되는 핵심 단어(예: 'macintosh' vs 'pentium')에 모델이 더 집중하게 만들어야 합니다.

3. 문맥 기반 임베딩(BERT 등)으로의 확장

- Word2Vec 계열은 단어의 고정된 벡터만 사용합니다. 'Apple'이 과일인지 컴퓨터 회사인지 문맥에 따라 동적으로 파악하는 BERT나 RoBERTa와 같은 트랜스포머 모델을 도입한다면, 현재의 '하드웨어 간 간섭' 문제를 근본적으로 해결할 수 있을 것으로 기대됩니다.

# 회고   

1. 카테고리 분류 자체는 의의를 특별하게 인지하게 어려운 프로젝트라고 느낌. 텍스트 데이터 처리/학습 실습에 의의를 두고 진행

2. 1번의 연장으로, 최신 모델과 거리가 있는 고전적인 모델이라 어텐션/트랜스포머 개념과 혼동 없이 이해를 할 필요 있을 것으로 보임.   
또한, 고전 모델 자체가 가지는 한계를 고려하여 성능 지표 개선을 추가로 진행하는 것은 리소스 낭비라고 판단하여 패스하였음.

3. 카테고리 분류가 아닌, 언어 이해/생성으로 진행 시 모델에 대한 평가 방법론을 고민해봤지만 딱히 방법이 떠오르지 않았음. (같은 맥락, 동일한 뜻이지만 다른 텍스트의 평가 등) 추후 스터디/실습을 통해 익혀야 할 것으로 보임.

4. 학습 모델의 성능, 임베딩 모델 파라미터 설정, 데이터 전처리중 어느 지점의 영향인지는 모르겠으나 임베딩 모델 별 평가지표가 유의미한 차이를 보이지 않았음. 모델 구현 방식에 따른 차이를 결론으로 제안했으나 정량적 지표와 유의미한 연결점을 보인다고 판단하기는 어려움.